In [1]:
from datetime import datetime
import time
import pandas as pd
import pickle as pk
from bs4 import BeautifulSoup
import requests

In [5]:
with open('condo_links_all.txt') as f:
    condo_links_all = f.read().splitlines()
print(f"Total condo links: {len(condo_links_all)}")

Total condo links: 111314


####500 link 45sec####

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import concurrent.futures

# Scraper class to manage the session and retrieval
class Scraper:
    def __init__(self):
        self.session = requests.Session()  # Reuse connections for efficiency

    def retrieve(self, link):
        try:
            page = self.session.get(link)
            soup = BeautifulSoup(page.content, 'html.parser')

            # Extracting the sqm container
            sqm_container = soup.find('div', class_="sc-ejnaz6-21 dWMjEs")
            if sqm_container:
                spans = sqm_container.find_all('span')
                last_span = spans[-1].get_text().strip() if spans else "No sqm data"
            else:
                last_span = "No sqm data"

            # Extract other information
            name_elem = soup.find(class_="sc-ejnaz6-2 fuLHNZ")
            name = name_elem.find(class_="sc-ejnaz6-3 gSIBgi").get_text() if name_elem else "No name"

            price_elem = soup.find(class_="sc-ejnaz6-5 hgrkiv")
            price = price_elem.find(class_="sale-price").get_text().strip('฿').replace(',', "") if price_elem else "No price"

            return (name, last_span, price)
        except Exception as e:
            print(f"Error processing {link}: {e}")
            return None

# Function to process links using threading
def process_links(links, max_workers=20):
    scraper = Scraper()
    condo_list = []
    total_links = len(links)
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_link = {executor.submit(scraper.retrieve, link): link for link in links}
        for i, future in enumerate(concurrent.futures.as_completed(future_to_link), 1):
            data = future.result()
            if data is not None:
                condo_list.append(data)
            if i % 500 == 0:  # Print progress every 1000 links
                print(f"Processed {i}/{total_links} links", end='\r', flush=True)

    print(f"Processing complete. Total links processed: {total_links}")
    return condo_list

# Function to read links from a text file
def read_links_from_file(file_path):
    with open(file_path, 'r') as file:
        links = [line.strip() for line in file.readlines()]
    return links

# Main function to execute the scraping and save the results
def main():
    file_path = 'condo_links_all.txt'
    condo_links_all = read_links_from_file(file_path)

    start_time = datetime.now()

    # Process the links and get the condo details
    condo_list = process_links(condo_links_all)

    # Create a DataFrame and save to CSV
    df = pd.DataFrame(condo_list, columns=['Name', 'SQM', 'Price'])
    df.to_csv("NewPrice.csv", header=['Name', 'SQM', 'Price'], index=False, encoding='utf-8-sig')

    print(f"Data saved, total valid entries: {len(condo_list)}")
    print(f'Time elapsed (hh:mm:ss.ms): {datetime.now() - start_time}')

if __name__ == "__main__":
    main()


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import concurrent.futures
import time

class Scraper:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})

    def retrieve(self, link):
        retries = 3
        for attempt in range(retries):
            try:
                page = self.session.get(link, timeout=10)
                page.raise_for_status()  # Raise an error for bad responses
                soup = BeautifulSoup(page.content, 'html.parser')

                # Use get() method to simplify extraction with defaults
                sqm_container = soup.find('div', class_="sc-ejnaz6-21 dWMjEs")
                last_span = sqm_container.find_all('span')[-1].get_text().strip() if sqm_container else "No sqm data"

                name_elem = soup.select_one(".sc-ejnaz6-2 .sc-ejnaz6-3")
                name = name_elem.get_text() if name_elem else "No name"

                price_elem = soup.select_one(".sc-ejnaz6-5 .sale-price")
                price = price_elem.get_text().strip('฿').replace(',', "") if price_elem else "No price"

                return (name, last_span, price)
            except requests.RequestException as e:
                print(f"Request error for {link}: {e}")
                break  # Exit on request error
            except Exception as e:
                print(f"Error processing {link}, Attempt {attempt+1}: {e}")
                if attempt < retries - 1:
                    time.sleep(1)  # Wait before retrying
                else:
                    return None

def process_links(links, max_workers=20):
    scraper = Scraper()
    condo_list = []
    total_links = len(links)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_link = {executor.submit(scraper.retrieve, link): link for link in links}
        for i, future in enumerate(concurrent.futures.as_completed(future_to_link), 1):
            data = future.result()
            if data is not None:
                condo_list.append(data)
            if i % 100 == 0:
                print(f"Processed {i}/{total_links} links", end='\r', flush=True)

    print(f"\nProcessing complete. Total links processed: {total_links}")
    return condo_list

def read_links_from_file(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file if line.strip()]  # Filter out empty lines

def main():
    file_path = 'condo_links_all.txt'
    condo_links_all = read_links_from_file(file_path)

    start_time = datetime.now()
    condo_list = process_links(condo_links_all)

    df = pd.DataFrame(condo_list, columns=['Name', 'SQM', 'Price'])
    df.to_csv("NewPrice.csv", header=['Name', 'SQM', 'Price'], index=False, encoding='utf-8-sig')

    print(f"Data saved, total valid entries: {len(condo_list)}")
    print(f'Time elapsed (hh:mm:ss.ms): {datetime.now() - start_time}')

if __name__ == "__main__":
    main()


Request error for https://propertyhub.in.th/en/listings/by-owner-niche-mono-sukhumvit-bearing-1-bedroom-floor20---4199070: HTTPSConnectionPool(host='propertyhub.in.th', port=443): Read timed out. (read timeout=10)
Request error for https://propertyhub.in.th/en/listings/2-br-penthouse-at-aritier-penthouse-at-ari-near-bts-ari-id-1793125---4654098: HTTPSConnectionPool(host='propertyhub.in.th', port=443): Read timed out. (read timeout=10)


In [3]:
pip install tqdm


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install aiohttp

Note: you may need to restart the kernel to use updated packages.


In [2]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import nest_asyncio  # For environments like Jupyter notebooks

# Apply nest_asyncio to allow nested event loops (only if needed in environments like Jupyter)
nest_asyncio.apply()

# Asynchronous Scraper class
class AsyncScraper:
    def __init__(self):
        self.session = None
    
    async def fetch(self, session, url):
        retries = 3
        for attempt in range(retries):
            try:
                async with session.get(url, timeout=10) as response:
                    response.raise_for_status()
                    html = await response.text()
                    soup = BeautifulSoup(html, 'lxml')

                    # Extracting required data
                    sqm_container = soup.find('div', class_="sc-ejnaz6-21 dWMjEs")
                    last_span = sqm_container.find_all('span')[-1].get_text().strip() if sqm_container else "No sqm data"

                    name_elem = soup.select_one(".sc-ejnaz6-2 .sc-ejnaz6-3")
                    name = name_elem.get_text() if name_elem else "No name"

                    price_elem = soup.select_one(".sc-ejnaz6-5 .sale-price")
                    price = price_elem.get_text().strip('฿').replace(',', "") if price_elem else "No price"

                    return (name, last_span, price)
            except (aiohttp.ClientError, asyncio.TimeoutError) as e:
                print(f"Error fetching {url}, attempt {attempt + 1}: {e}")
                if attempt < retries - 1:
                    await asyncio.sleep(2 ** attempt)  # Exponential backoff
                else:
                    return None

    async def scrape_links(self, urls, max_concurrency=20):
        async with aiohttp.ClientSession(headers={'User-Agent': 'Mozilla/5.0'}) as session:
            tasks = []
            semaphore = asyncio.Semaphore(max_concurrency)  # Limit concurrency

            async def bound_fetch(url):
                async with semaphore:
                    return await self.fetch(session, url)

            tasks = [bound_fetch(url) for url in urls]
            results = await asyncio.gather(*tasks)
            
            return [result for result in results if result is not None]

# Helper function to read links from file
def read_links_from_file(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file if line.strip()]

# Main function to execute the scraping
async def run_scraper():
    file_path = 'condo_links_all1.txt'
    condo_links_all = read_links_from_file(file_path)
    scraper = AsyncScraper()

    start_time = datetime.now()

    # Scrape the links
    condo_list = await scraper.scrape_links(condo_links_all)

    # Save to DataFrame and CSV
    df = pd.DataFrame(condo_list, columns=['Name', 'SQM', 'Price'])
    df.to_csv("NewPrice3.csv", header=['Name', 'SQM', 'Price'], index=False, encoding='utf-8-sig')

    print(f"Data saved, total valid entries: {len(condo_list)}")
    print(f'Time elapsed (hh:mm:ss.ms): {datetime.now() - start_time}')

# Function to check if the event loop is already running and run the scraper
def main():
    try:
        # If already inside an event loop (e.g., Jupyter Notebook), use this
        if asyncio.get_event_loop().is_running():
            return asyncio.ensure_future(run_scraper())
        # Otherwise, use asyncio.run() to run the main async function
        else:
            return asyncio.run(run_scraper())
    except RuntimeError:
        # For environments like Jupyter Notebook, where get_event_loop() may raise an error
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        loop.run_until_complete(run_scraper())

# Run the scraper
if __name__ == "__main__":
    main()


Error fetching https://propertyhub.in.th/en/listings/condo-for-sale-whizdom-the-forestias-petopia-pet-live-here-suitable-for-pet-owners-with-30-rai-forest-garden-view-5-51-mb---4622199, attempt 1: 
Error fetching https://propertyhub.in.th/en/listings/sale-624-condolette-line-id-0619636888---4461480, attempt 1: 
Data saved, total valid entries: 500
Time elapsed (hh:mm:ss.ms): 0:00:31.811962


In [6]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
from tqdm import tqdm  # Progress bar
import nest_asyncio  # For environments like Jupyter notebooks

# Apply nest_asyncio to allow nested event loops (only if needed in environments like Jupyter)
nest_asyncio.apply()

# Asynchronous Scraper class
class AsyncScraper:
    def __init__(self):
        self.session = None
    
    async def fetch(self, session, url):
        retries = 3
        for attempt in range(retries):
            try:
                async with session.get(url, timeout=10) as response:
                    response.raise_for_status()
                    html = await response.text()
                    soup = BeautifulSoup(html, 'lxml')

                    # Extracting required data
                    sqm_container = soup.find('div', class_="sc-ejnaz6-21 dWMjEs")
                    last_span = sqm_container.find_all('span')[-1].get_text().strip() if sqm_container else "No sqm data"

                    name_elem = soup.select_one(".sc-ejnaz6-2 .sc-ejnaz6-3")
                    name = name_elem.get_text() if name_elem else "No name"

                    price_elem = soup.select_one(".sc-ejnaz6-5 .sale-price")
                    price = price_elem.get_text().strip('฿').replace(',', "") if price_elem else "No price"

                    return (name, last_span, price)
            except (aiohttp.ClientError, asyncio.TimeoutError) as e:
                print(f"Error fetching {url}, attempt {attempt + 1}: {e}")
                if attempt < retries - 1:
                    await asyncio.sleep(2 ** attempt)  # Exponential backoff
                else:
                    return None

    async def scrape_links(self, urls, max_concurrency=20):
        async with aiohttp.ClientSession(headers={'User-Agent': 'Mozilla/5.0'}) as session:
            tasks = []
            semaphore = asyncio.Semaphore(max_concurrency)  # Limit concurrency

            async def bound_fetch(url):
                async with semaphore:
                    return await self.fetch(session, url)

            tasks = [bound_fetch(url) for url in urls]
            
            # Using tqdm to show progress
            results = []
            for future in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Processing"):
                result = await future
                if result:
                    results.append(result)

            return results

# Helper function to read links from file
def read_links_from_file(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file if line.strip()]

# Main function to execute the scraping
async def run_scraper():
    file_path = 'condo_links_all.txt'
    condo_links_all = read_links_from_file(file_path)
    scraper = AsyncScraper()

    start_time = datetime.now()

    # Scrape the links with progress bar
    condo_list = await scraper.scrape_links(condo_links_all)

    # Save to DataFrame and CSV
    df = pd.DataFrame(condo_list, columns=['Name', 'SQM', 'Price'])
    df.to_csv("propertynewprice.csv", header=['Name', 'SQM', 'Price'], index=False, encoding='utf-8-sig')

    print(f"Data saved, total valid entries: {len(condo_list)}")
    print(f'Time elapsed (hh:mm:ss.ms): {datetime.now() - start_time}')

# Function to check if the event loop is already running and run the scraper
def main():
    try:
        # If already inside an event loop (e.g., Jupyter Notebook), use this
        if asyncio.get_event_loop().is_running():
            return asyncio.ensure_future(run_scraper())
        # Otherwise, use asyncio.run() to run the main async function
        else:
            return asyncio.run(run_scraper())
    except RuntimeError:
        # For environments like Jupyter Notebook, where get_event_loop() may raise an error
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        loop.run_until_complete(run_scraper())

# Run the scraper
if __name__ == "__main__":
    main()


Processing:   0%|          | 243/111314 [00:25<2:31:34, 12.21it/s]

Error fetching https://propertyhub.in.th/en/listings/lumpini-ville-sukhumvit-76-bearing-station--9ddb08f8---4170046, attempt 1: 


Processing:   0%|          | 269/111314 [00:27<1:27:15, 21.21it/s]

Error fetching https://propertyhub.in.th/en/listings/1-br-condo-at-noble-revo-silom-near-bts-surasak-id-1552914---4529381, attempt 1: 


Processing:   1%|          | 754/111314 [01:03<3:32:10,  8.68it/s]

...

Processing:  39%|███▉      | 43726/111314 [1:01:35<1:24:17, 13.37it/s]

Error fetching https://propertyhub.in.th/en/listings/1-br-condo-at-euro-condominium-close-to-phratamnak-id-1177052---4531428, attempt 1: 


Processing:  39%|███▉      | 43861/111314 [1:01:49<1:41:02, 11.13it/s]

Error fetching https://propertyhub.in.th/en/listings/s-fcst105-condo-for-sale-fuse-chan-sathorn-6th-floor-33-sq-m-1-bedroom-1-bathroom-3-2-million-091-942-6249---4438977, attempt 1: 


Processing:  39%|███▉      | 43922/111314 [1:01:56<2:08:33,  8.74it/s]